# TPC3: Processador de Pessoas listadas nos Róis de Confessados

(publicado em **2023.02.28**)

Construa agora um ou vários programas Python
para  processar o texto 'processos.txt' (procurar o ficheiro no Bb) com o intuito de
calcular frequências de alguns elementos (a ideia é utilizar arrays associativos, dicionários em Python,
para o efeito) conforme solicitado a seguir:

> a) Calcula a frequência de processos por ano (primeiro elemento da data);

> b) Calcula a frequência de nomes próprios (o primeiro em cada nome) e apelidos (o ultimo em cada nome) por séculos e apresenta os 5 mais usados;

> c) Calcula a frequência dos vários tipos de relação: irmão, sobrinho, etc.;

> d) Converta os 20 primeiros registos num novo ficheiro de output mas em formato **Json**.
